In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from scipy import stats
from geopy.distance import great_circle
from scipy.stats.stats import pearsonr

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:

station = 'USW00013967'   
#station = 'USW00093009'
 
filename = '/Users/rphinney/Documents/Hollings/ghcnd_all/'+station+'.dly'
file = open(filename, 'r')


In [4]:

    
#Using ghcnd-stations.txt get the station information

path = '/Users/rphinney/Documents/Hollings/ghcnd-stations.txt'

station_file = open(path)
ghcnd = []

for x in station_file:
    ghcnd.append(x)

for each_line in ghcnd:   
    if each_line[0:11] == station:
        station_name = each_line[40:70]
        station_state = each_line[38:40]
        latitude = each_line[13:20]
        longitude = each_line[21:31]
        elevation = each_line[32:38]
        station_name.strip()
        print(station_name)
        print(latitude)
        print(longitude)
            
            
            
#------------------------------------------------------------------
#find nearest grid from the livneh file
    
filepath = '/Users/rphinney/Documents/Hollings/livneh_NAmerExt_15Oct2014_pr-above-99th_historical_1950_2013.nc'
data = Dataset(filepath)
    
lon = data.variables['lon'][:]
lat = data.variables['lat'][:]
print(data.variables.keys())
prcp = data.variables['consecDD'][:]

            
station_loc = (latitude,longitude)
min_loc = 9999
    
for lons in lon:
    for lats in lat:
        check_loc = (lats,lons)
        b = int(great_circle(station_loc,check_loc).miles)
        if b < min_loc:
            min_loc = b
            min_lat = lats
            min_lon = lons-360

y = np.arange(1950,2014)           
#scatter plot the precip values at the grid nearest to the lincoln station
prcpa = []
lon_count = -1
lat_count = -1
for lons in lon:
    lon_count = lon_count +1
    if lons == min_lon+360:
        for lats in lat:
            lat_count = lat_count + 1
            if lats == min_lat:
                for x in range (0,64):
                    prcp2 = prcp[x,lat_count,lon_count]
                    prcpa.append(prcp2)
    
                    #plt.plot(y[x],.03937*prcp[x,lat_count,lon_count],'o', color = 'blue')


y = np.arange(1950,2014)
#plt.plot(y,prcpa,color = 'blue')

plt.title('Annual Maximum Consecutive Dry Days')
plt.suptitle(station_name.strip() + ' VS LIVNEH NEAREST GRID BOX')


#---------------------------------------------------------------------------
#---------------------------------------------------------------------------



begin_year = 1950
end_year = 2013

mylist = []
max_wet = []

for line in file:
    mylist.append(line)
    


#print(min_year)
max_year = 2017
#day_max = 0
day_maxa = []      #array of max precip for one day each year
five_max = []
max_dry = []
max_wet = []
for year in range(1950,2014):
    year_prcp = []
    for line in mylist:
        if int(line[11:15]) == year:
            if line[17:21] == 'PRCP':
                year_prcp.append(line)

    new_prcp = []
    trace = []
    for data in year_prcp:
        char1 = 22
        char2 = 26
        while char1 != 270:
            if data[char1:char2] == '9999':
                char1 = char1 + 8
                char2 = char2 + 8
            else:
                prcp = (.003937*int(data[char1:char2])) 
                new_prcp.append(round(prcp,2))
                mflag = data[char2:char2+1]
                trace.append(mflag)
                char1 = char1 + 8
                char2 = char2 + 8
    
    #Annal max consecutive dry days-------------------------------------------------
    count_dry = 0
    dry_days = 0

    for b in range(0,size(new_prcp)):
        value_d = new_prcp[b]
        if value_d == 0:
            if trace[b] == 'T':
                dry_days = max(count_dry, dry_days)
                count_dry = 0
            else:
                count_dry = count_dry + 1
        if value_d > 0:
            dry_days = max(count_dry, dry_days)
            count_dry = 0           
    if dry_days == 0:
        dry_days = -9999
    max_dry.append(dry_days)

    
x = np.arange(1950,2013)
#plt.plot(x,daymax, color = 'red')


def zero_to_nan(values):
    """Replace every 0 with 'nan' and return a copy."""
    return [float('nan') if x==0 else x for x in values]



plt.scatter(annual,prcpa)

pearson = pearsonr(annual,prcpa)
          
plt.figtext(.15,.8, 'Pearson Correlation: '+ str(round(pearson[0],3)))

 OKLAHOMA CITY WILL ROGERS AP 
35.3889
 -97.6006 
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_above_99th'])


KeyError: 'consecDD'